# NLP learning for bot (conservative twitter bot)

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import seaborn as cb
import json
import numpy as np
import nltk 
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import re
import matplotlib.pyplot as plt
import itertools
import collections

## Data scrapping

In [ ]:
tweets_con = []

#scrape data and append tweets to list
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('#conservative').get_items()): # declare a username
  if i>2000: #number of tweets you want to scrape
    break
  tweets_con.append([tweet.content]) # declare the attributes to be returned

# creating a dataframe from the list
tweets_con_df = pd.DataFrame(tweets_con, columns=['Text'])

### Creating json of tweets for preservation

In [2]:
file = open("tweet_con.json",'w')
j = json.dumps(tweets_con, default = str)
file.write(j)
file.close()

NameError: name 'tweets_con' is not defined

### Clean up

In [ ]:
def clean_text(txt):
    cleaned = re.sub(r"http\S+","", txt)
    return cleaned

In [ ]:
cleaned_text = [clean_text(elements) for elements in tweets_con_df.Text]
tt = TweetTokenizer()
normalize = [tt.tokenize(elements.lower()) for elements in cleaned_text]

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words.add('#conservative')

# remove stop words and punctuation
complete = [[w for w in words if not w in stop_words if w.isalnum()] for words in normalize]